# A/B Test Analysis with spark-bestfit (RayBackend)

This notebook demonstrates how to analyze **A/B test results** using statistical
distribution fitting with proper uncertainty quantification.

## What You'll Learn

1. **Model conversion rates** with bounded Beta distributions
2. **Quantify uncertainty** using bootstrap confidence intervals
3. **Analyze revenue per user** with mixed discrete-continuous data
4. **Calculate statistical significance** and practical effect sizes

## Business Context

A/B testing is fundamental to data-driven product development:

- **Feature launches**: Does the new checkout flow improve conversion?
- **Pricing experiments**: How does a 10% price change affect revenue?
- **UX optimization**: Which button color drives more clicks?

## Prerequisites

```bash
pip install spark-bestfit[ray] pandas numpy matplotlib scipy
```

## Setup

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

import ray
from spark_bestfit import DistributionFitter
from spark_bestfit.backends.ray import RayBackend

# Initialize Ray (skip if already initialized)
if not ray.is_initialized():
    ray.init(ignore_reinit_error=True)

# Create RayBackend
backend = RayBackend()
print(f"RayBackend initialized with {backend.get_parallelism()} CPUs")

## Part 1: Simulate A/B Test Data

We'll simulate a website conversion experiment:

- **Control (A)**: Current checkout page - 8% conversion rate
- **Treatment (B)**: New checkout design - 9.5% conversion rate (true lift)

In [ ]:
np.random.seed(42)

# Experiment parameters
n_control = 5000
n_treatment = 5000
conv_rate_control = 0.08
conv_rate_treatment = 0.095

# Generate conversion events
control_conversions = np.random.binomial(1, conv_rate_control, n_control)
treatment_conversions = np.random.binomial(1, conv_rate_treatment, n_treatment)

# Generate revenue
def generate_revenue(conversions, mean_aov=75, sigma=0.6):
    revenue = np.zeros(len(conversions))
    converted_mask = conversions == 1
    n_converted = converted_mask.sum()
    mu = np.log(mean_aov) - (sigma**2 / 2)
    revenue[converted_mask] = np.random.lognormal(mu, sigma, n_converted)
    return revenue

control_revenue = generate_revenue(control_conversions)
treatment_revenue = generate_revenue(treatment_conversions, mean_aov=78)

print("Experiment Summary:")
print(f"  Control: {n_control} visitors, {control_conversions.sum()} conversions ({control_conversions.mean():.2%})")
print(f"  Treatment: {n_treatment} visitors, {treatment_conversions.sum()} conversions ({treatment_conversions.mean():.2%})")
print(f"  Observed lift: {(treatment_conversions.mean() / control_conversions.mean() - 1):.1%}")

In [ ]:
# Visualize the experiment data
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Conversion rates
rates = [control_conversions.mean() * 100, treatment_conversions.mean() * 100]
bars = axes[0].bar(['Control', 'Treatment'], rates, color=['steelblue', 'coral'], edgecolor='black')
axes[0].set_ylabel('Conversion Rate (%)')
axes[0].set_title('Conversion Rates')
axes[0].set_ylim(0, 15)
for bar, rate in zip(bars, rates):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3, f'{rate:.2f}%', ha='center')

# Revenue distribution
control_rev_converted = control_revenue[control_revenue > 0]
treatment_rev_converted = treatment_revenue[treatment_revenue > 0]
axes[1].hist(control_rev_converted, bins=30, alpha=0.6, label='Control', color='steelblue', density=True)
axes[1].hist(treatment_rev_converted, bins=30, alpha=0.6, label='Treatment', color='coral', density=True)
axes[1].set_xlabel('Revenue ($)')
axes[1].set_ylabel('Density')
axes[1].set_title('Revenue Distribution (Converted Users)')
axes[1].legend()

# Revenue per visitor
rpv_control = control_revenue.mean()
rpv_treatment = treatment_revenue.mean()
bars = axes[2].bar(['Control', 'Treatment'], [rpv_control, rpv_treatment], color=['steelblue', 'coral'], edgecolor='black')
axes[2].set_ylabel('Revenue per Visitor ($)')
axes[2].set_title('Revenue per Visitor')

plt.tight_layout()
plt.show()

## Part 2: Model Conversion Rates with Beta Distribution

For conversion rates (proportions between 0 and 1), the **Beta distribution** is ideal.

In [ ]:
# Bootstrap conversion rate samples
def bootstrap_conversion_rates(conversions, n_bootstrap=1000, sample_size=500, seed=42):
    np.random.seed(seed)
    rates = []
    for _ in range(n_bootstrap):
        sample = np.random.choice(conversions, size=sample_size, replace=True)
        rates.append(sample.mean())
    return np.array(rates)

control_rates = bootstrap_conversion_rates(control_conversions)
treatment_rates = bootstrap_conversion_rates(treatment_conversions)

control_rates_df = pd.DataFrame({'conversion_rate': control_rates})
treatment_rates_df = pd.DataFrame({'conversion_rate': treatment_rates})

print(f"Bootstrap samples: {len(control_rates)} rates per group")

In [ ]:
# Fit bounded distributions
fitter = DistributionFitter(backend=backend)

control_results = fitter.fit(
    control_rates_df,
    column='conversion_rate',
    bounded=True,
    lower_bound=0.0,
    upper_bound=1.0,
    max_distributions=10,
    lazy_metrics=True
)

treatment_results = fitter.fit(
    treatment_rates_df,
    column='conversion_rate',
    bounded=True,
    lower_bound=0.0,
    upper_bound=1.0,
    max_distributions=10,
    lazy_metrics=True
)

control_fit = control_results.best(n=1, metric='aic')[0]
treatment_fit = treatment_results.best(n=1, metric='aic')[0]

print(f"Control: {control_fit.distribution}, AIC={control_fit.aic:.2f}")
print(f"Treatment: {treatment_fit.distribution}, AIC={treatment_fit.aic:.2f}")

## Part 3: Bootstrap Confidence Intervals

In [ ]:
def confidence_interval(data, confidence=0.95):
    alpha = 1 - confidence
    return np.percentile(data, alpha/2 * 100), np.percentile(data, (1 - alpha/2) * 100)

# Calculate CIs
control_ci = confidence_interval(control_rates)
treatment_ci = confidence_interval(treatment_rates)
lift_samples = (treatment_rates - control_rates) / control_rates
lift_ci = confidence_interval(lift_samples)
diff_samples = treatment_rates - control_rates
diff_ci = confidence_interval(diff_samples)

print("95% Confidence Intervals:")
print(f"\nControl: {control_rates.mean():.4f} [{control_ci[0]:.4f}, {control_ci[1]:.4f}]")
print(f"Treatment: {treatment_rates.mean():.4f} [{treatment_ci[0]:.4f}, {treatment_ci[1]:.4f}]")
print(f"Relative Lift: {lift_samples.mean()*100:.1f}% [{lift_ci[0]*100:.1f}%, {lift_ci[1]*100:.1f}%]")
print(f"\nP(Treatment > Control): {(diff_samples > 0).mean():.1%}")

## Part 4: Revenue Analysis

In [ ]:
# Fit revenue distributions
control_rev = control_revenue[control_revenue > 0]
treatment_rev = treatment_revenue[treatment_revenue > 0]

control_rev_results = fitter.fit(
    pd.DataFrame({'revenue': control_rev}),
    column='revenue',
    max_distributions=15,
    lazy_metrics=True
)

treatment_rev_results = fitter.fit(
    pd.DataFrame({'revenue': treatment_rev}),
    column='revenue',
    max_distributions=15,
    lazy_metrics=True
)

print("Revenue Distribution Fits:")
print(f"\nControl - Best by AIC:")
for fit in control_rev_results.best(n=3, metric='aic'):
    print(f"  {fit.distribution}: AIC={fit.aic:.1f}")
print(f"\nTreatment - Best by AIC:")
for fit in treatment_rev_results.best(n=3, metric='aic'):
    print(f"  {fit.distribution}: AIC={fit.aic:.1f}")

## Part 5: Business Impact Projection

In [ ]:
# Bootstrap RPV
def bootstrap_rpv(conversions, revenue, n_bootstrap=1000, seed=42):
    np.random.seed(seed)
    rpvs = []
    n = len(conversions)
    for _ in range(n_bootstrap):
        idx = np.random.choice(n, size=n, replace=True)
        rpvs.append(revenue[idx].mean())
    return np.array(rpvs)

control_rpv = bootstrap_rpv(control_conversions, control_revenue)
treatment_rpv = bootstrap_rpv(treatment_conversions, treatment_revenue)
rpv_diff = treatment_rpv - control_rpv

# Annual impact
annual_visitors = 1_000_000
annual_impact = rpv_diff * annual_visitors
annual_impact_ci = confidence_interval(annual_impact)

print(f"Projected Annual Revenue Impact:")
print(f"  Expected: ${annual_impact.mean():,.0f}")
print(f"  95% CI: [${annual_impact_ci[0]:,.0f}, ${annual_impact_ci[1]:,.0f}]")
print(f"  P(Positive): {(annual_impact > 0).mean():.1%}")

## Part 6: Decision Report

In [ ]:
min_acceptable_lift = 0.05
implementation_cost = 50_000
rpv_lift = (treatment_rpv - control_rpv) / control_rpv
prob_min_lift = (rpv_lift >= min_acceptable_lift).mean()
prob_negative = (annual_impact < 0).mean()

print("="*60)
print("A/B TEST DECISION REPORT")
print("="*60)
print(f"\n KEY METRICS:")
print(f"   Conversion Rate Lift: {lift_samples.mean()*100:.1f}%")
print(f"   RPV Lift: {rpv_lift.mean()*100:.1f}%")
print(f"\n RISK ASSESSMENT:")
print(f"   P(Negative Impact): {prob_negative:.1%}")
print(f"   P(Lift >= 5%): {prob_min_lift:.1%}")
print(f"\n RECOMMENDATION:")
if prob_negative < 0.05 and prob_min_lift > 0.80:
    print("   LAUNCH - Strong evidence of positive impact")
elif prob_negative < 0.10:
    print("   CONSIDER LAUNCHING - Moderate evidence")
else:
    print("   CONTINUE TESTING")
print("="*60)

## Summary

This notebook demonstrated A/B test analysis using RayBackend for distributed processing.

### Key Features Used

| Feature | Purpose |
|---------|----------|
| `RayBackend` | Distributed parallel processing |
| `bounded=True` | Fit Beta to [0, 1] proportions |
| `lazy_metrics=True` | Fast model selection |
| `results.best(n=3)` | Compare top candidates |

In [ ]:
# Cleanup
# ray.shutdown()  # Uncomment to shutdown Ray when done
print("A/B test analysis complete!")